Denne fil bliver brugt til at analysere og kigge på det datasæt vi har valgt at lave vores endelige projekt i. 



In [9]:
import kagglehub
import os
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("dsabljic/movielens-32m")
print("Path to dataset files:", path)

# List subfolder 'ml-32m'
dataset_folder = os.path.join(path, "ml-32m")
print("Indhold i mappen ml-32m:")
print(os.listdir(dataset_folder))

# Find en csv-fil i mappen
csv_files = [f for f in os.listdir(dataset_folder) if f.endswith('.csv')]



# Læs første CSV-fil (bare som eksempel)
if csv_files:
    links_csv = os.path.join(dataset_folder, csv_files[0])
    df = pd.read_csv(links_csv)

    movies_csv = os.path.join(dataset_folder, "movies.csv")
    df_movies = pd.read_csv(movies_csv)

    ratings_csv = os.path.join(dataset_folder, "ratings.csv")
    df_ratings = pd.read_csv(ratings_csv)

    tags_csv = os.path.join(dataset_folder, "tags.csv")
    df_tags = pd.read_csv(tags_csv)
    
else:
    print("Ingen CSV-filer fundet i mappen.")


Path to dataset files: C:\Users\eriks\.cache\kagglehub\datasets\dsabljic\movielens-32m\versions\1
Indhold i mappen ml-32m:
['checksums.txt', 'links.csv', 'movies.csv', 'ratings.csv', 'README.txt', 'tags.csv']
